# Importing Modules

In [1]:
# Importing dataframe and storage modules
import pandas as pd
import numpy as np

# Importing os, json, and time module
import os, json, time

# Importing SQL modules
import pymysql
pymysql.install_as_MySQLdb()
from sqlalchemy import create_engine

# Importing matplotlib and seaborn modules
import matplotlib.pyplot as plt
import seaborn as sns

# Importing tmdb module
import tmdbsimple as tmdb
from tqdm.notebook import tqdm_notebook as tqdm

In [2]:
basic = 'https://datasets.imdbws.com/title.basics.tsv.gz'
akas = 'https://datasets.imdbws.com/title.akas.tsv.gz'
rating = 'https://datasets.imdbws.com/title.ratings.tsv.gz'

In [3]:
basics = pd.read_csv(basic, sep='\t', low_memory=False)
akas_df = pd.read_csv(akas, sep='\t', low_memory=False)
ratings = pd.read_csv(rating, sep='\t', low_memory=False)

# Filtering/Cleaning

- For this project I want to break down each dataset into 3 parts. This will allow me to make the code more readable for the user.

Now I will create a custom function to evaluate imported dataframes using different Pandas methods, this will allow me to view the data for future proccessing.

In [4]:
# Creating a function that evaluates a given dataset
def eval_df(df):
    print(df.head())
    print(f'Dataset Information Eval: \n{df.info(memory_usage=True, show_counts=True)}\n')
    print(f'Columns with NULL values: \n{df.isna().sum()}\n')
    print(f'Duplicated: \n{df.duplicated().sum()}')

# Title Basics

- I will now use the function created from earlier to get more insight on the basics dataframe.

In [5]:
# Calling the function to evaluate the dataset
eval_df(basics)

      tconst titleType            primaryTitle           originalTitle  \
0  tt0000001     short              Carmencita              Carmencita   
1  tt0000002     short  Le clown et ses chiens  Le clown et ses chiens   
2  tt0000003     short          Pauvre Pierrot          Pauvre Pierrot   
3  tt0000004     short             Un bon bock             Un bon bock   
4  tt0000005     short        Blacksmith Scene        Blacksmith Scene   

  isAdult startYear endYear runtimeMinutes                    genres  
0       0      1894      \N              1         Documentary,Short  
1       0      1892      \N              5           Animation,Short  
2       0      1892      \N              4  Animation,Comedy,Romance  
3       0      1892      \N             12           Animation,Short  
4       0      1893      \N              1              Comedy,Short  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9684449 entries, 0 to 9684448
Data columns (total 9 columns):
 #   Column      

In [6]:
# Using the value_counts() method to get the number of unique values in the column
basics.value_counts()

tconst      titleType  primaryTitle                                      originalTitle                                     isAdult  startYear  endYear  runtimeMinutes  genres                    
tt0000001   short      Carmencita                                        Carmencita                                        0        1894       \N       1               Documentary,Short             1
tt27074922  movie      The Official SammyClassicSonicFan Roblox Movie 2  The Official SammyClassicSonicFan Roblox Movie 2  0        2015       \N       \N              Adventure                     1
tt27074899  tvEpisode  Episode #1.4674                                   Episode #1.4674                                   0        1974       \N       \N              Drama,Romance                 1
tt2707490   tvEpisode  The 25th Anniversary Spectacular                  The 25th Anniversary Spectacular                  0        2013       \N       \N              Comedy                        1
tt270

1. Replace "\N" with np.nan.

In [7]:
# Using the replace() method to replace \N with np.nan
basics.replace({'\\N':np.nan}, inplace=True)
basics.isin(['\\N']).sum()

tconst            0
titleType         0
primaryTitle      0
originalTitle     0
isAdult           0
startYear         0
endYear           0
runtimeMinutes    0
genres            0
dtype: int64

2. Eliminate movies that are null for runtimeMinutes
3. Eliminate movies that are null for genres

I decided to drop both of these columns within one dropna() method.

In [8]:
# Using the dropna() method to drop rows with null values
basics.dropna(subset=['runtimeMinutes', 'genres'], inplace=True)
# Checking the number of null values in the dataset
print(basics.isna().sum())

tconst                  0
titleType               0
primaryTitle            1
originalTitle           1
isAdult                 0
startYear          158533
endYear           2713526
runtimeMinutes          0
genres                  0
dtype: int64


4. Filter column titleType values to movie.

In [9]:
# Filtering titleType column to get only movies
titleType = basics['titleType'] == 'movie'
basics_f = basics.loc[titleType]
# Checking dataframe to see if the filter worked
basics_f.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
8,tt0000009,movie,Miss Jerry,Miss Jerry,0,1894,NaN,45,Romance
144,tt0000147,movie,The Corbett-Fitzsimmons Fight,The Corbett-Fitzsimmons Fight,0,1897,NaN,100,"Documentary,News,Sport"
570,tt0000574,movie,The Story of the Kelly Gang,The Story of the Kelly Gang,0,1906,NaN,70,"Action,Adventure,Biography"
587,tt0000591,movie,The Prodigal Son,L'enfant prodigue,0,1907,NaN,90,Drama
672,tt0000679,movie,The Fairylogue and Radio-Plays,The Fairylogue and Radio-Plays,0,1908,NaN,120,"Adventure,Fantasy"


5. Filter column startYear values between 2000-2022.

In [10]:
# Filtering startYear column to get only movies released after 2000
df_filter = basics_f['startYear'] >= '2000'
basics_f = basics_f.loc[df_filter]
# Checking dataframe to see if the filter worked
basics_f.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
13082,tt0013274,movie,Istoriya grazhdanskoy voyny,Istoriya grazhdanskoy voyny,0,2021,NaN,133,Documentary
34803,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,NaN,118,"Comedy,Fantasy,Romance"
61116,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020,NaN,70,Drama
67669,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018,NaN,122,Drama
76059,tt0077684,movie,Histórias de Combóios em Portugal,Histórias de Combóios em Portugal,0,2022,NaN,46,Documentary


6. Eliminate movies that include "Documentary" in genre.

In [11]:
# Filtering genres column to get only movies that are not documentaries
is_doc = basics_f['genres'].str.contains('documentary', case=False)
basics_f = basics_f[~is_doc]
# Checking if the filter worked
print(basics_f['genres'].loc[basics_f['genres']=='Documentary'].sum())

0


----

# AKAs

- First before the cleaning process I want to use the custom function created earlier to get some information on this data.

In [12]:
eval_df(akas_df)

     titleId  ordering                      title region language  \
0  tt0000001         1                 Карменсіта     UA       \N   
1  tt0000001         2                 Carmencita     DE       \N   
2  tt0000001         3  Carmencita - spanyol tánc     HU       \N   
3  tt0000001         4                 Καρμενσίτα     GR       \N   
4  tt0000001         5                 Карменсита     RU       \N   

         types     attributes isOriginalTitle  
0  imdbDisplay             \N               0  
1           \N  literal title               0  
2  imdbDisplay             \N               0  
3  imdbDisplay             \N               0  
4  imdbDisplay             \N               0  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35230595 entries, 0 to 35230594
Data columns (total 8 columns):
 #   Column           Non-Null Count     Dtype 
---  ------           --------------     ----- 
 0   titleId          35230595 non-null  object
 1   ordering         35230595 non-null

In [13]:
akas_df.value_counts()

titleId     ordering  title             region  language  types        attributes  isOriginalTitle
tt0000001   1         Карменсіта        UA      \N        imdbDisplay  \N          0                  1
tt26074272  7         Folge #1.2786     DE      de        \N           \N          0                  1
tt26074292  7         Épisode #1.2789   FR      fr        \N           \N          0                  1
            6         एपिसोड #1.2789    IN      hi        \N           \N          0                  1
            5         Episodio #1.2789  ES      es        \N           \N          0                  1
                                                                                                     ..
tt13839326  6         エピソード #1.280      JP      ja        \N           \N          0                  1
            5         Episódio #1.280   PT      pt        \N           \N          0                  1
            4         एपिसोड #1.280     IN      hi        \N         

1. Keep only US movies from the region column.

In [14]:
region_f = akas_df['region'] == 'US'
akas_df = akas_df.loc[region_f]
akas_df.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
5,tt0000001,6,Carmencita,US,\N,imdbDisplay,\N,0
14,tt0000002,7,The Clown and His Dogs,US,\N,\N,literal English title,0
33,tt0000005,10,Blacksmith Scene,US,\N,imdbDisplay,\N,0
36,tt0000005,1,Blacksmithing Scene,US,\N,alternative,\N,0
41,tt0000005,6,Blacksmith Scene #1,US,\N,alternative,\N,0


2. Replace "\N" with np.nan

In [15]:
akas_df = akas_df.replace({'\\N':np.nan})
akas_df.isin(['\\N']).sum()

titleId            0
ordering           0
title              0
region             0
language           0
types              0
attributes         0
isOriginalTitle    0
dtype: int64

----

# Ratings

Again, I will be using eval_df() to evaulate the ratings dataframe.

In [16]:
eval_df(ratings)

      tconst  averageRating  numVotes
0  tt0000001            5.7      1959
1  tt0000002            5.8       263
2  tt0000003            6.5      1795
3  tt0000004            5.6       179
4  tt0000005            6.2      2596
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1288262 entries, 0 to 1288261
Data columns (total 3 columns):
 #   Column         Non-Null Count    Dtype  
---  ------         --------------    -----  
 0   tconst         1288262 non-null  object 
 1   averageRating  1288262 non-null  float64
 2   numVotes       1288262 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 29.5+ MB
Dataset Information Eval: 
None

Columns with NULL values: 
tconst           0
averageRating    0
numVotes         0
dtype: int64

Duplicated: 
0


In [17]:
ratings.value_counts()

tconst     averageRating  numVotes
tt0000001  5.7            1959        1
tt2219622  5.7            655         1
tt2219680  9.0            9           1
tt2219676  3.4            15          1
tt2219674  5.7            34          1
                                     ..
tt0936227  6.8            10          1
tt0936226  7.7            11          1
tt0936225  7.2            11          1
tt0936224  6.8            10          1
tt9916880  8.2            6           1
Length: 1288262, dtype: int64

1. Replace "/N" with np.nan (if any).

In [18]:
ratings = ratings.replace({'\\N':np.nan})
ratings.isin(['\\N']).sum()

tconst           0
averageRating    0
numVotes         0
dtype: int64

# Other filtering

Now I will filter both dataframes basics and ratings to only have movies produced in this US.

In [19]:
keepers = basics['tconst'].isin(akas_df['titleId'])
basics_f = basics_f[keepers]
basics_f.head()

C:\Users\zachd\AppData\Local\Temp\ipykernel_23932\751135143.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  basics_f = basics_f[keepers]


,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
34803,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,NaN,118,"Comedy,Fantasy,Romance"
61116,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020,NaN,70,Drama
67669,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018,NaN,122,Drama
86801,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005,NaN,100,"Comedy,Horror,Sci-Fi"
93938,tt0096056,movie,Crime and Punishment,Crime and Punishment,0,2002,NaN,126,Drama


In [20]:
keepers = ratings['tconst'].isin(akas_df['titleId'])
ratings = ratings[keepers]
ratings.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,1959
1,tt0000002,5.8,263
4,tt0000005,6.2,2596
5,tt0000006,5.1,177
6,tt0000007,5.4,815


# Storing Dataframes Filtered

In [21]:
# Using make_dirs() method to create a directory
os.makedirs('Data/', exist_ok=True)
# Using the listdir() method to list the files in the directory
os.listdir("Data/")

['final_tmdb_data_2000.csv.gz',
 'final_tmdb_data_2001.csv.gz',
 'title_akas_df.cvs.gz',
 'title_basics_f.cvs.gz',
 'title_ratings.cvs.gz',
 'tmdb_api_results_2000.json',
 'tmdb_api_results_2001.json']

In [22]:
# Using the to_csv() method to save the dataframe as a csv file
basics_f.to_csv("Data/title_basics_f.cvs.gz", compression='gzip', index=False)
# Checking if csv file was created
basics = pd.read_csv("Data/title_basics_f.cvs.gz", low_memory=False)
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,NaN,118,"Comedy,Fantasy,Romance"
1,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020,NaN,70,Drama
2,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018,NaN,122,Drama
3,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005,NaN,100,"Comedy,Horror,Sci-Fi"
4,tt0096056,movie,Crime and Punishment,Crime and Punishment,0,2002,NaN,126,Drama


In [23]:
# Using the to_csv() method to save the dataframe as a csv file
akas_df.to_csv("Data/title_akas_df.cvs.gz", compression='gzip', index=False)
# Checking if csv file was created
akas_df = pd.read_csv("Data/title_akas_df.cvs.gz", low_memory=False)
akas_df.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,6,Carmencita,US,NaN,imdbDisplay,NaN,0.0
1,tt0000002,7,The Clown and His Dogs,US,NaN,NaN,literal English title,0.0
2,tt0000005,10,Blacksmith Scene,US,NaN,imdbDisplay,NaN,0.0
3,tt0000005,1,Blacksmithing Scene,US,NaN,alternative,NaN,0.0
4,tt0000005,6,Blacksmith Scene #1,US,NaN,alternative,NaN,0.0


In [24]:
# Using the to_csv() method to save the dataframe as a csv file
ratings.to_csv("Data/title_ratings.cvs.gz", compression='gzip', index=False)
# Checking if csv file was created
ratings = pd.read_csv("Data/title_ratings.cvs.gz", low_memory=False)
ratings.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,1959
1,tt0000002,5.8,263
2,tt0000005,6.2,2596
3,tt0000006,5.1,177
4,tt0000007,5.4,815


- Before saving, run a final .info() for each of the dataframes to show a summary of how many movies remain and the datatypes of each feature

In [25]:
print(basics.info())
print(akas_df.info())
print(ratings.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87052 entries, 0 to 87051
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   tconst          87052 non-null  object 
 1   titleType       87052 non-null  object 
 2   primaryTitle    87052 non-null  object 
 3   originalTitle   87052 non-null  object 
 4   isAdult         87052 non-null  int64  
 5   startYear       87052 non-null  int64  
 6   endYear         0 non-null      float64
 7   runtimeMinutes  87052 non-null  int64  
 8   genres          87052 non-null  object 
dtypes: float64(1), int64(3), object(5)
memory usage: 6.0+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1421195 entries, 0 to 1421194
Data columns (total 8 columns):
 #   Column           Non-Null Count    Dtype  
---  ------           --------------    -----  
 0   titleId          1421195 non-null  object 
 1   ordering         1421195 non-null  int64  
 2   title            1421

In [26]:
FOLDER = 'Data/'
os.makedirs(FOLDER, exist_ok=True)
os.listdir(FOLDER)

['final_tmdb_data_2000.csv.gz',
 'final_tmdb_data_2001.csv.gz',
 'title_akas_df.cvs.gz',
 'title_basics_f.cvs.gz',
 'title_ratings.cvs.gz',
 'tmdb_api_results_2000.json',
 'tmdb_api_results_2001.json']

# Using TMDB API to compare IMDB Movies

- I will be using TMDB API to extract data from their database to compare against the IMDB movies extracted from earlier data cleanign processes.

> **NOTE:** This will only include movies made in the US.

In [27]:
# Using open() method to open .sercet file and load json file containing api key for tmdb
with open(r"C:\Users\zachd\.secret\tmdb_api.json", 'r') as f:
    api_key = json.load(f) # Loading json file with json module

tmdb.API_KEY = api_key['api-key'] # Setting the api key to the tmdb module

In [28]:
## Creating a function that takes in a movie id and returns a dictionary with the movie info and rating
def get_movie_with_rating(movie_id):
    # get the movie object for the current id
    movie = tmdb.Movies(movie_id)
    # save the .info .releases dictionaries
    info = movie.info()
    releases = movie.releases()
    # Loop through countries in releases
    for c in releases['countries']:
        # if the country abbreviation==US
        if c['iso_3166_1']=='US':
            ## save a "certification" key in info with the certification
            info['certification'] = c['certification']

    return info

In [29]:
# Creating a function that takes the new data and the filename and appends the new data to a json file
def write_json(new_data, filename):
    """Appends a list of records (new_data) to a JSON file (filename).
    Adapted from: https://www.geeksforgeeks.org/append-to-a-json-file-using-python/
    """
    with open(filename, 'r+') as file:
        # First we load existing data into a dict.
        file_data = json.load(file)
        ## choose extend or append
        if (type(new_data) == list) & (type(file_data) == list):
            file_data.extend(new_data)
        else:
            file_data.append(new_data)
        # Sets file's current position at offset.
        file.seek(0)
        # convert back to json.
        json.dump(file_data, file)

In [30]:
# Using the read_csv() method to read the csv file into a dataframe
basics = pd.read_csv("Data/title_basics_f.cvs.gz", low_memory=False)
# Seting the index for YEARS column
YEARS_TO_GET = [2000, 2001]
# Creating empty list to store the errors
errors = []
# Creating a for loop to iterate through the years
for YEAR in tqdm(YEARS_TO_GET, desc='YEARS', position=0):
    # Creating a variable to store the json file name
    JSON_FILE = f'{FOLDER}tmdb_api_results_{YEAR}.json'
    # Checking if the json file exists
    file_exists = os.path.isfile(JSON_FILE)
    # If the file does not exist, create it and add an empty list
    if file_exists == False:
        with open(JSON_FILE, 'w') as file:
            json.dump([{'imdb_id':0}], file)
    # Creating a variable to store the dataframe filtered by the year
    df = basics.loc[basics['startYear']==YEAR].copy()
    # Creating a variable to store the movie ids
    movie_ids = df['tconst'].copy()
    # Loading existing data from json into a daraframe called "previous_df"
    previous_df = pd.read_json(JSON_FILE)
    # Filter out any ids that are already in the JSON_FILE
    movie_ids_to_get = movie_ids[~movie_ids.isin(previous_df['imdb_id'])]
    # Get index and movie id from list
    for movie_id in tqdm(movie_ids_to_get, desc=f'Movies from {YEAR}', position=1, leave=True):
        try:
            # Rtrieve then data for the movie id
            temp = get_movie_with_rating(movie_id)
            # Append/extend results to existing file using a pre-made function
            write_json(temp, JSON_FILE)
            # Short 20 ms sleep to prevent overwhelming server
            time.sleep(0.2)
        except Exception as e:
            errors.append([movie_id, e])

    # Read in the json file as a dataframe
    final_yead_df = pd.read_json(JSON_FILE)
    # Save the dataframe as a csv file
    final_yead_df.to_csv(f'{FOLDER}final_tmdb_data_{YEAR}.csv.gz', compression="gzip", index=False)

print(f'- Number of errors: {len(errors)}')

YEARS:   0%|          | 0/2 [00:00<?, ?it/s]

Movies from 2000:   0%|          | 0/210 [00:00<?, ?it/s]

Movies from 2001:   0%|          | 0/245 [00:00<?, ?it/s]

- Number of errors: 455


In [31]:
# Using the to_csv() method to save the dataframe as a csv file
final_year_2000 = pd.read_csv("Data/final_tmdb_data_2000.csv.gz", low_memory=False)
print(final_year_2000.head())
print(final_year_2000.info())

     imdb_id  adult                     backdrop_path belongs_to_collection  \
0          0    NaN                               NaN                   NaN   
1  tt0113026    0.0  /vMFs7nw6P0bIV1jDsQpxAieAVnH.jpg                   NaN   
2  tt0113092    0.0                               NaN                   NaN   
3  tt0116391    0.0                               NaN                   NaN   
4  tt0118694    0.0  /n4GJFGzsc7NinI1VeGDXIcQjtU2.jpg                   NaN   

       budget                                             genres homepage  \
0         NaN                                                NaN      NaN   
1  10000000.0  [{'id': 35, 'name': 'Comedy'}, {'id': 10402, '...      NaN   
2         0.0           [{'id': 878, 'name': 'Science Fiction'}]      NaN   
3         0.0  [{'id': 18, 'name': 'Drama'}, {'id': 28, 'name...      NaN   
4    150000.0  [{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...      NaN   

         id original_language   original_title  ...     revenu

In [32]:
# Using the to_csv() method to save the dataframe as a csv file
final_year_2001 = pd.read_csv("Data/final_tmdb_data_2001.csv.gz", low_memory=False)
print(final_year_2001.head())
print(final_year_2001.info())

     imdb_id  adult                     backdrop_path belongs_to_collection  \
0          0    NaN                               NaN                   NaN   
1  tt0035423    0.0  /hfeiSfWYujh6MKhtGTXyK3DD4nN.jpg                   NaN   
2  tt0114447    0.0                               NaN                   NaN   
3  tt0118589    0.0  /9NZAirJahVilTiDNCHLFcdkwkiy.jpg                   NaN   
4  tt0118652    0.0  /mWxJEFRMvkG4UItYJkRDMgWQ08Y.jpg                   NaN   

       budget                                             genres homepage  \
0         NaN                                                NaN      NaN   
1  48000000.0  [{'id': 10749, 'name': 'Romance'}, {'id': 14, ...      NaN   
2         0.0  [{'id': 53, 'name': 'Thriller'}, {'id': 28, 'n...      NaN   
3  22000000.0  [{'id': 18, 'name': 'Drama'}, {'id': 10402, 'n...      NaN   
4   1000000.0  [{'id': 27, 'name': 'Horror'}, {'id': 9648, 'n...      NaN   

         id original_language         original_title  ...     